In [ ]:
import json

IT_train_path = 'IT_train.json'
IT_test_path = 'IT_test.json'
CL_train_path = 'CL_train.json'
CL_test_path = 'CL_test.json'

train_it = {}
test_it = {}
train_cl = {}
test_cl = {}

with open(IT_train_path, 'r') as f:
    train_it = json.load(f)
    f.close()
    
with open(IT_test_path, 'r') as f:
    test_it = json.load(f)
    f.close()

with open(CL_train_path, 'r') as f:
    train_cl = json.load(f)
    f.close()

with open(CL_test_path, 'r') as f:
    test_cl = json.load(f)
    f.close()
    

## Preparing Data in txt format(sentence tab label)

In [ ]:
train_path_it = 'handcrafted_features/train_it/'
train_path_cl = 'handcrafted_features/train_cl/'
test_path_it = 'handcrafted_features/test_it/'
test_path_cl = 'handcrafted_features/test_cl/'

In [ ]:
for case in train_it.keys():
    sentences = train_it[case]['sentences']
    labels = train_it[case]['complete']
    all_text = ""
    for sent, label in zip(sentences, labels):
        all_text += sent+"\t"+label+"\n"
    with open(train_path_it+case, "w") as f:
        f.write(all_text)
        f.close()

In [ ]:
for case in train_cl.keys():
    sentences = train_cl[case]['sentences']
    labels = train_cl[case]['complete']
    all_text = ""
    for sent, label in zip(sentences, labels):
        all_text += sent+"\t"+label+"\n"
    with open(train_path_cl+case, "w") as f:
        f.write(all_text)
        f.close()

In [ ]:
for case in test_it.keys():
    sentences = test_it[case]['sentences']
    labels = test_it[case]['complete']
    all_text = ""
    for sent, label in zip(sentences, labels):
        all_text += sent+"\t"+label+"\n"
    with open(test_path_it+case, "w") as f:
        f.write(all_text)
        f.close()

In [ ]:
for case in test_cl.keys():
    sentences = test_cl[case]['sentences']
    labels = test_cl[case]['complete']
    all_text = ""
    for sent, label in zip(sentences, labels):
        all_text += sent+"\t"+label+"\n"
    with open(test_path_cl+case, "w") as f:
        f.write(all_text)
        f.close()

In [ ]:
import os
import numpy as np
from tqdm import tqdm

## Getting Cue-NE features

In [ ]:
path = "handcrafted_features/train_cl/"
pathw = "handcrafted_features/train_cl_ne/"
fr_ne = open("handcrafted-features-code/NE.txt","r")
fr_cue = open("handcrafted-features-code/cue_phrases_saravanan.txt","r")

In [ ]:

hcf = []
#count = {}
print ("pos tags")
for line in fr_ne.readlines():
    line = line.rstrip("\n")
    hcf.append(line)

for line in fr_cue.readlines():
    line = line.rstrip("\n")
    hcf.append(line)
print(hcf)
print ("file reading starts")
ctr = 0
for file in tqdm(os.listdir(path)):
    url = os.path.join(path,file)
    fr = open(url,"r")
    ctr+=1
    #print (url)
    fw = open(os.path.join(pathw,file),"w")
    
    for line in fr.readlines():
        labels = np.zeros(len(hcf),dtype='int32')
        line = line.rstrip("\n\r")
        ls = line.split("\t")
        sent = ls[0]
        lab = ls[1]
        for c in hcf:
            if c in line:
                ind = hcf.index(c)
                labels[ind] = labels[ind]+1
                
        sum=0
        for i in labels:
            sum+=i
        print(labels)
        fw.write(sent+"$$$"+lab+"$$$")
        for i in labels:
            try:
                fw.write(str(round(float(i)/float(sum),5))+"$$$")
                
            except:
                fw.write("0.0"+"$$$")
        fw.write("\n")
    fw.close()   
    fr.close()     

## Getting Layout Features

In [ ]:
def get_para_position(original_docs_path):
    file_para_pos = {}
    
    for file in tqdm(os.listdir(original_docs_path)):
        fr = open(os.path.join(original_docs_path,file),"r")
        count=0
        para_pos = {}
        for line in fr.readlines():
            line = line.rstrip("\n")
            line = line.split("\t")[0]
            count+=1
            para_pos [line] = count
        
        for k,v in para_pos.items():
            norm_v = round(float(v)/float(count),5)
            para_pos[k] = norm_v
            
        file_para_pos[file] = para_pos
    
    return file_para_pos


def get_sentence_position(sent_split_docs_path):
    file_sent_pos = {}
    
    for file in tqdm(os.listdir(sent_split_docs_path)):
        fr = open(os.path.join(sent_split_docs_path,file),"r")
        count=0
        sent_pos = {}
        for line in fr.readlines():
            line = line.rstrip("\n")
            count+=1
            sent_pos [line] = count
        
        for k,v in sent_pos.items():
            norm_v = round(float(v)/float(count),5)
            sent_pos[k] = norm_v
            
        file_sent_pos[file] = sent_pos
        
    return file_sent_pos
  
path_whole = "handcrafted_features/test_cl/"
path_ss = "handcrafted_features/test_cl/"
pathw = "handcrafted_features/test_cl_layout/"

print("\n\n para pos \n\n")
para_pos = get_para_position(path_whole)

print("\n\n sent pos \n\n")
sent_pos = get_sentence_position(path_ss)

for file, sent_features in sent_pos.items():
    fw = open(os.path.join(pathw,file),"w")
    for line,sent_feats in sent_features.items():
        line,label = line.split("\t")
        for para, para_feats in para_pos[file].items():
            if line in para:
                para_feat = para_feats
                break
        fw.write(line+"$$$"+label+"$$$"+str(para_feat)+"$$$"+str(sent_feats)+"\n")
    fw.close()

## Getting POS features

In [ ]:
import nltk
import os
import numpy as np
from tqdm import tqdm

path = "handcrafted_features/train_cl/"
pathw = "handcrafted_features/train_cl_pos/"
fr_pos = open("handcrafted-features-code/postags_list.txt","r")

postags = []
#count = {}
print ("pos tags")
for line in fr_pos.readlines():
    line = line.rstrip("\n")
    postags.append(line)

print ("file reading starts")
ctr = 0
for file in tqdm(os.listdir(path)):
    url = os.path.join(path,file)
    fr = open(url,"r")
    ctr+=1
    print (url)
    fw = open(os.path.join(pathw,file),"w")
    
    for line in fr.readlines():
        labels = np.zeros(len(postags),dtype='int32')
        line = line.rstrip("\n\r")
        ls = line.split("\t")
        sent = ls[0]
        lab = ls[1]
        tokens = nltk.word_tokenize(sent)
        pos_list = nltk.pos_tag(tokens)
        
        for p in pos_list:
            pos = p[1]
            if pos in postags:
                ind = postags.index(pos)
                labels[ind] = labels[ind]+1
        sum=0
        for i in labels:
            sum+=i
        
        fw.write(sent+"$$$"+lab+"$$$")
        for i in labels:
            try:
                fw.write(str(round(float(i)/float(sum),5))+"$$$")
            except:
                fw.write("0.0"+"$$$")
        fw.write("\n")
        
    fw.close()   
    fr.close()     

In [ ]:
nltk.download('averaged_perceptron_tagger')

## Appending all the 3 features

In [ ]:
import os

layout_feats = "handcrafted_features/test_cl_layout/"
pos_feats = "handcrafted_features/test_cl_pos/"
cue_ne_feats = "handcrafted_features/test_cl_ne/"


pathw_818 = "handcrafted_features/test_cl_final/"


for file in os.listdir(layout_feats):
    fr_lay = open(os.path.join(layout_feats,file),"r")
    fr_pos = open(os.path.join(pos_feats,file),"r")
    fr_cuene =  open(os.path.join(cue_ne_feats,file),"r")
    
    fw_818 = open(os.path.join(pathw_818,file),"w")

    
    lays = fr_lay.readlines()
    pos = fr_pos.readlines()
    cuene = fr_cuene.readlines()
    
    for i in range(len(lays)):
        item1 = lays[i]
        item2 = pos[i]
        item3 = cuene[i]
        
        item1 = item1.rstrip("\n")
        item2 = item2.rstrip("\n")
        item3 = item3.rstrip("\n")

        ls1 = item1.split("$$$")
        ls2 = item2.split("$$$")
        ls3 = item3.split("$$$")
        
        sent = ls1[0].lstrip(" ").rstrip(" ")
        lab = ls1[1].lstrip(" ").rstrip(" ")
        
        
        f1 = ls1[2:]
        f2 = ls2[2:]
        f3 = ls3[2:]
        
        line  = sent+"$$$"+lab+"$$$"+"$$$".join(f1)+"$$$"+"$$$".join(f2)+"$$$"+"$$$".join(f3)+"\n"
        fw_818.write(sent+"$$$"+lab+"$$$"+"$$$".join(f1)+"$$$"+"$$$".join(f2)+"$$$"+"$$$".join(f3)+"\n")

    fw_818.close()

## Getting final embeddings

In [ ]:
path_inp = "handcrafted_features/test_it_final/"
path_out = "handcrafted_features/test_it_handcrafted/"

In [ ]:
mapping = {}
mapping['ArgumentPetitioner'] = 'Argument'
mapping['ArgumentRespondent'] = 'Argument'
mapping['Fact'] = 'Fact'
mapping['Issue'] = 'Fact'
mapping['PrecedentNotReliedUpon'] = 'Precedent'
mapping['PrecedentReliedUpon'] = 'Precedent'
mapping['PrecedentOverruled'] = 'Precedent'
mapping['RatioOfTheDecision'] = 'RatioOfTheDecision'
mapping['Statute'] = 'Statute'
mapping['RulingByPresentCourt'] = 'RulingByPresentCourt'
mapping['RulingByLowerCourt'] = 'RulingByLowerCourt'

In [ ]:
for case in os.listdir(path_inp):
    all_text = ""
    with open(os.path.join(path_inp, case), "r") as f:
        for sent in f.readlines():
            line = sent.rstrip("$$$\n")
            tags = line.split("$$$")
            sentence = tags[0]
            label = str(tags[1])
            features = tags[2:]
            if(label == "Dissent" or label == "None"):
                continue
            all_text += (" ").join(features)+"\t"+mapping[label]+"\n" 
        f.close()
    with open(os.path.join(path_out, case), "w") as f:
        f.write(all_text)
        f.close()